# Main Configuration UI

<b>This notebook is not supposed to be used on its own.<b>

In [ ]:
from exasol.utils import upward_file_search

# This NB may be running from various locations in the NB hierarchy.
# Need to search for other supporting NBs from the current directory upwards.

%run {upward_file_search('utils/ui_styles.ipynb')}
%run {upward_file_search('utils/generic_config_ui.ipynb')}

In [ ]:
from exasol.ai_lab_config import AILabConfig as Cfg
import ipywidgets as widgets


def get_db_selection_ui() -> widgets.Widget:
    """
    Creates a UI form for choosing between the ITDE and a customer database.

    There should be a global variable `sb_config` referencing a configuration store object.
    """

    ui_look = get_config_styles()

    db_options = ['ITDE', 'Custom']
    db_choice = 0 if sb_config.get(Cfg.use_itde.value, 'True') == 'True' else 1
    db_selector = widgets.RadioButtons(options=db_options, value=db_options[db_choice], 
                                       layout=ui_look.input_layout, style=ui_look.input_style)
    select_btn = widgets.Button(description='Select', style=ui_look.button_style, layout=ui_look.button_layout)
    header_lbl = widgets.Label(value='Database Choice', style=ui_look.header_style, layout=ui_look.header_layout)


    def select_database(btn):
        sb_config.save(Cfg.use_itde.value, str(db_selector.value == db_options[0]))
        btn.icon = 'check'

    def on_value_change(change):
        select_btn.icon = 'pen'

    select_btn.on_click(select_database)
    db_selector.observe(on_value_change, names=['value'])

    group_items = [header_lbl, widgets.Box([db_selector], layout=ui_look.row_layout)]
    items = [widgets.Box(group_items, layout=ui_look.group_layout), select_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    return ui


def get_custom_db_config_ui() -> widgets.Widget:
    """
    Creates a UI form for editing a customer database configuration.
    
    There should be a global variable `sb_config` referencing a configuration store object.
    """

    inputs = [
        [
            ('Host Name', widgets.Text(value=sb_config.get(Cfg.db_host_name.value, '127.0.0.1')), Cfg.db_host_name.value),
            ('Port', widgets.IntText(value=int(sb_config.get(Cfg.db_port.value, '8888'))), Cfg.db_port.value),
            ('User Name', widgets.Text(value=sb_config.get(Cfg.db_user.value)), Cfg.db_user.value),
            ('Password', widgets.Password(value=sb_config.get(Cfg.db_password.value)), Cfg.db_password.value),
            ('Default Schema', widgets.Text(value=sb_config.get(Cfg.db_schema.value, 'IDA')), Cfg.db_schema.value),
            ('Encrypted Comm.', widgets.Checkbox(value=sb_config.get(Cfg.db_encryption.value, 'True') == 'True', indent=False),
                Cfg.db_encryption.value)
        ],
        [
            ('Port', widgets.IntText(value=int(sb_config.get(Cfg.bfs_port.value, '6666'))), Cfg.bfs_port.value),
            ('User Name', widgets.Text(value=sb_config.get(Cfg.bfs_user.value)), Cfg.bfs_user.value),
            ('Password', widgets.Password(value=sb_config.get(Cfg.bfs_password.value)), Cfg.bfs_password.value),
            ('Service Name', widgets.Text(value=sb_config.get(Cfg.bfs_service.value, 'bfsdefault')), Cfg.bfs_service.value),
            ('Bucket Name', widgets.Text(value=sb_config.get(Cfg.bfs_bucket.value, 'default')), Cfg.bfs_bucket.value),
            ('Encrypted Comm.', widgets.Checkbox(value=sb_config.get(Cfg.bfs_encryption.value, 'True') == 'True', indent=False),
                Cfg.bfs_encryption.value)
        ],
        [
            ('Validate Certificate', widgets.Checkbox(value=sb_config.get(Cfg.cert_vld.value, 'True') == 'True', indent=False),
                Cfg.cert_vld.value),
            ('Trusted CA File/Dir', widgets.Text(value=sb_config.get(Cfg.trusted_ca.value)), Cfg.trusted_ca.value),
            ('Certificate File', widgets.Text(value=sb_config.get(Cfg.client_cert.value)), Cfg.client_cert.value),
            ('Private Key File', widgets.Text(value=sb_config.get(Cfg.client_key.value)), Cfg.client_key.value)
        ]
    ]

    group_names = ['Database Connection', 'BucketFS Connection', 'TLS/SSL Configuration']

    return get_generic_config_ui(sb_config, inputs, group_names)


def get_itde_config_ui() -> widgets.Widget:
    """
    Creates a UI form for editing ITDE configuration.

    There should be a global variable `sb_config` referencing a configuration store object.
    """

    inputs = [
        [
            ('Memory Size (GiB)', widgets.IntText(value=int(sb_config.get(Cfg.mem_size.value, '2'))), Cfg.mem_size.value),
            ('Disk Size (GiB)', widgets.IntText(value=int(sb_config.get(Cfg.disk_size.value, '2'))), Cfg.disk_size.value),
            ('Default Schema', widgets.Text(value=sb_config.get(Cfg.db_schema.value, 'IDA')), Cfg.db_schema.value)
        ],
    ]

    group_names = ['Database Configuration']

    return get_generic_config_ui(sb_config, inputs, group_names)



def get_config_ui() -> widgets.Widget:
    """
    Creates a db configuration UI, depending on the choice of the database.
    """

    if sb_config.get(Cfg.use_itde.value, 'True') == 'True':
        return get_itde_config_ui()
    else:
        return get_custom_db_config_ui()


def get_start_itde_ui() -> widgets.Widget:
    """
    A UI for starting or restarting the ITDE.
    It checks if an instance of the ITDE is already running. If it is, a warning is displayed.
    An existing instance will be shut down before a new one is created.

    There should be a global variable `sb_config` referencing a configuration store object.
    """

    if sb_config.get(Cfg.use_itde.value) != 'True':
        return None

    from exasol.itde_manager import (bring_itde_up, is_itde_running, take_itde_down)

    ui_look = get_config_styles()

    is_running = is_itde_running(sb_config)

    header_text = 'Restart ITDE' if is_running else 'Start ITDE'
    header_lbl = widgets.Label(value=header_text, style=ui_look.header_style, layout=ui_look.header_layout)
        
    run_btn = widgets.Button(description='Run', style=ui_look.button_style, layout=ui_look.button_layout)

    def run_itde(btn):
        if is_itde_running(sb_config):
            take_itde_down(sb_config)
        bring_itde_up(sb_config)
        btn.icon = 'check'

    run_btn.on_click(run_itde)

    group_items = [header_lbl]
    if is_running:
        warning_text = 'Please note that all data stored in the currently running instance of the database will be lost!'
        warning_html = widgets.HTML(value= '<style>p{word-wrap: break-word}</style><p>' + warning_text +' </p>')
        group_items.append(widgets.Box([warning_html], layout=ui_look.row_layout))
    items = [widgets.Box(group_items, layout=ui_look.group_layout), run_btn]
    ui = widgets.Box(items, layout=ui_look.outer_layout)
    return ui
